In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/m4_fmt_data.csv")
# sha_subset = pd.read_csv("/content/drive/My Drive/sha_subset.csv")
# paper = pd.read_csv("/content/drive/My Drive/metadata_2021-04-26.csv")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import roc_auc_score

,Unnamed: 0,hadm_id,note_id,chartdate,charttime,text,hospital_expire_flag
0,0,22595853,10000032-RR-14,2180-05-06 21:19:00,2180-05-06 21:19:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,0
1,1,22595853,10000032-RR-15,2180-05-06 23:00:00,2180-05-06 23:00:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...,0
2,2,22595853,10000032-RR-16,2180-05-07 09:55:00,2180-05-07 09:55:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi...",0
3,3,22841357,10000032-RR-22,2180-06-26 17:15:00,2180-06-26 17:15:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...,0
4,4,22841357,10000032-RR-23,2180-06-26 17:17:00,2180-06-26 17:17:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,0
...,...,...,...,...,...,...,...
1144753,1144753,23865745,19999987-RR-17,2145-11-02 22:37:00,2145-11-02 22:37:00,"HISTORY: ___, with left occipital bleeding. ...",0
1144754,1144754,23865745,19999987-RR-18,2145-11-03 04:35:00,2145-11-03 04:35:00,INDICATION: ___ female intubated for head ble...,0
1144755,1144755,23865745,19999987-RR-19,2145-11-03 16:40:00,2145-11-03 16:40:00,HISTORY: ___ woman with left occipital hemorr...,0
1144756,1144756,23865745,19999987-RR-20,2145-11-04 05:10:00,2145-11-04 05:10:00,PORTABLE CHEST OF ___\n\nCOMPARISON: ___ radi...,0


In [ ]:
from nltk.corpus import stopwords
import nltk
import re
from bs4 import BeautifulSoup
nltk.download('stopwords')
eng_stopwords= set(stopwords.words('english'))
def clean_text(text):
  text = BeautifulSoup(text, 'html.parser').get_text()
  text = re.sub(r'[^a-zA-Z]', ' ', text)
  words = text.lower().split()
  words = [w for w in words if w not in eng_stopwords]
  return ' '.join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data['clean_text'] = data['text'].apply(clean_text)

<ipython-input-4-573e71f534e7>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [ ]:
abstract = data['clean_text'].values
label = data['hospital_expire_flag'].values

In [ ]:
abstract = abstract[:10000]
label = label[:10000]

In [ ]:
data1 = {
    'text': abstract,
    'hospital_expire_flag': label
}

df = pd.DataFrame(data1)

In [ ]:
class ClinicalDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = self.labels[index]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
MAX_LEN = 512

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df.text.tolist(),
    df.hospital_expire_flag.tolist(),
    test_size=0.1,
    random_state=42
)

train_dataset = ClinicalDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = ClinicalDataset(val_texts, val_labels, tokenizer, MAX_LEN)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [ ]:
BATCH_SIZE = 16

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)
model.to('cuda')  # 将模型移动到 GPU

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.332000,0.290368
100,0.314600,0.271128
150,0.242400,0.281182
200,0.270600,0.285242
250,0.229200,0.270909
300,0.353100,0.260961
350,0.246800,0.251031
400,0.328200,0.250009
450,0.280500,0.248539
500,0.304900,0.237282


TrainOutput(global_step=1689, training_loss=0.22801809723099142, metrics={'train_runtime': 3784.2994, 'train_samples_per_second': 7.135, 'train_steps_per_second': 0.446, 'total_flos': 7103998494720000.0, 'train_loss': 0.22801809723099142, 'epoch': 3.0})

In [ ]:
val_predictions = trainer.predict(val_dataset)
val_labels = val_predictions.label_ids
val_preds = val_predictions.predictions.argmax(-1)

# 计算 AUROC
val_probs = torch.nn.functional.softmax(torch.tensor(val_predictions.predictions), dim=-1)[:, 1].cpu().numpy()
auroc = roc_auc_score(val_labels, val_probs)

print(f"Validation AUROC: {auroc}")

Validation AUROC: 0.8600620870357712


In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.2053249031305313, 'eval_runtime': 33.7983, 'eval_samples_per_second': 29.587, 'eval_steps_per_second': 1.864, 'epoch': 3.0}
